<a href="https://colab.research.google.com/github/helenatong/oc_p4_Anticipez_les_besoins_en_consommation_de_batiments/blob/main/entrainement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

---

## **Prérequis**
- Télécharger [le zip](https://drive.google.com/file/d/1Hw7gjd0cZQGi258_aosxPfC-RTSkcqJt/view?usp=sharing) dans Google Drive
- Modifier les chemins des fichiers (zip_path et extract_path) dans le code


---

 ## **Contexte et description du notebook**

L'objectif de Seattle est de devenir une ville neutre en émissions de carbone en 2050. La ville s'intéresse particulièrement aux consommations énergétiques et aux émissions de carbone des bâtiments qui ne sont pas destinés à l'habitation.

Néanmoins, ces 2 types de données ne sont pas disponibles pour tous les bâtiments. L'objectif ici est de prédire la consommation énergétique et l'émission de carbone.

De plus, un focus sera effectué sur si la variable ENERGYSTARScore Score joue un rôle important dans la précision des prédictions des émissions de carbone.

Les variables TotalGHGEmissions et SiteEnergyUseWN ont été choisies pour respectivement mesurer la consommation énergétique et l'émission de carbone.

---

## **Sommaire**

### **I. Installation et mise en place de l'environnement de travail**

### **II. Définitions des fonctions auxiliaires**

### **III. Analyse exploratoire**

# **I. Installation et mise en place de l'environnement de travail**

In [20]:
# Importation des librairies
import numpy as np
import pandas as pd

import zipfile
import os

import re

import sys

import missingno as msno
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression

import datetime

In [21]:
# Informations sur l'environnement virtuel
print('Version des librairies :')
print('Python        : ' + sys.version)
print('NumPy         : ' + np.version.full_version)
print('Pandas        : ' + pd.__version__)
print('Matplotlib    : ' + mpl.__version__)
print('Seaborn       : ' + sns.__version__)

now  = datetime.datetime.now().isoformat()
print('Run date      : ' + now)

Version des librairies :
Python        : 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
NumPy         : 1.25.2
Pandas        : 2.0.3
Matplotlib    : 3.7.1
Seaborn       : 0.13.1
Run date      : 2024-04-27T16:55:48.994544


In [22]:
# Configurations de l'affichages des données
pd.set_option('display.max_columns', None)

In [23]:
# Connexion au drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Verifier l'importation
df = pd.read_parquet('/content/drive/MyDrive/openclassrooms/project_list/project_4/data/cleaned_2016_Building_Energy_Benchmarking.pq')
df.head(2)

,OSEBuildingID,BuildingType,PrimaryPropertyType,Neighborhood,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEnergyUse(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions,Age,LastYearENERGYSTARCertified,NbYearENERGYSTARCertified
0,1,NonResidential,Hotel,DOWNTOWN,1.0,12,88434,0,88434,Hotel,88434.0,None,NaN,None,NaN,60.0,7226362.5,2003882.0,3946027.0,1276453.0,249.98,89,NaN,0
1,2,NonResidential,Hotel,DOWNTOWN,1.0,11,103566,15064,88502,Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,61.0,8387933.0,0.0,3242851.0,5145082.0,295.86,20,NaN,0


# **II. Définitions des fonctions et des variables globales**


In [25]:
# TARGET =
ID = 'OSEBuildingID'

In [26]:
# préparation des données pour ML, variable a de l'impact ou pas?
 # normalisation des données
 # encodage
 # dataset de training

 # TARGET 1 - prédire une valeur numérique
#tester simple la régression lineaire

#NEW NOTEBOOK
# TARGET 1 - prédire une valeur numérique
#tester simple la régression lineaire

# entrainement modèle, pipeline, grid search
# choix de la méthode de scoring
# comparer les 4 modèles : ElasticNet, SVM, GradientBoosting, RandomForest
# si iverfitting : tester une méthode ensembliste (bagging, boosting)

# check impact de STAR SCORING avec et sans
#c'est pour comparer ce qui est comparable. ENtre BTP aux services similaires (école <> industrie)

#NEW NOTEBOOK
# TARGET 2 - prédire une valeur numérique
# entrainement modèle

#shap
#ElasticNet (lineaire, ), SVM, GradientBoosting, RandomForest).

#pour savoir si une va est importante ?
# simple : utiliser la lib black box
# compliqué : LASSO (connaissance de l'algo!), cas particulier de l'eslatic net, 0 sur tout

#**Entrainement du modèle**


In [27]:
# On choisie toutes les variables numériques et 3 variables catégorielles
nb_null_value = df.isna().sum(axis=0)
col_to_keep = nb_null_value[nb_null_value < 100].index.to_list()
num_column = df.dtypes[(df.dtypes == 'float64') | (df.dtypes == 'int64')].index.to_list()
cat_to_keep = ['PrimaryPropertyType', 'Neighborhood']
num_to_keep = list(set(col_to_keep).intersection(num_column))
FINAL_COL =  num_to_keep + cat_to_keep #set pour le non itérable, #list pour concat ensuite
FINAL_COL = ['TotalGHGEmissions', 'Electricity(kBtu)', 'SiteEnergyUse(kBtu)', 'NaturalGas(kBtu)'
             , 'NumberofBuildings', 'PrimaryPropertyType', 'Neighborhood']
print((FINAL_COL))

['TotalGHGEmissions', 'Electricity(kBtu)', 'SiteEnergyUse(kBtu)', 'NaturalGas(kBtu)', 'NumberofBuildings', 'PrimaryPropertyType', 'Neighborhood']


In [28]:
TARGET = 'TotalGHGEmissions'
FEATURE = [col for col in FINAL_COL if col != TARGET]
df_train = df[FINAL_COL].dropna()
print(FEATURE)

['Electricity(kBtu)', 'SiteEnergyUse(kBtu)', 'NaturalGas(kBtu)', 'NumberofBuildings', 'PrimaryPropertyType', 'Neighborhood']


In [29]:
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import OneHotEncoder, PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline

In [30]:
preprocessor = ColumnTransformer(
    transformers=[
        ('encoder_nominal', OneHotEncoder(handle_unknown='ignore',sparse_output=False), cat_to_keep),
         ('norma', PowerTransformer(), ['Electricity(kBtu)', 'SiteEnergyUse(kBtu)', 'NaturalGas(kBtu)', 'NumberofBuildings'])
    ],
    remainder='drop'
  )

In [31]:
pp = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Lasso()),
])

In [32]:
# On va séparer les variables explicatives de la variable cible.
y = df_train.pop(TARGET)
X = df_train
print(y.shape, X.shape)
# # Question : encodage sur tout le jeu de données ou une partie seulement ? a priori separer puis encoder car pbm de type
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# X_train_df = pd.DataFrame(data=X_train, columns=FEATURE)
# y_train_df = pd.DataFrame(data=y_train, columns=[TARGET])

# model = PowerTransformer()
# y_train_normed = model.fit_transform(y_train_df)
X_df = pd.DataFrame(data=X, columns=FEATURE)
y_df = pd.DataFrame(data=y, columns=[TARGET])
pp.fit(X, y)

(1476,) (1476, 6)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('encoder_nominal',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['PrimaryPropertyType',
                                                   'Neighborhood']),
                                                 ('norma', PowerTransformer(),
                                                  ['Electricity(kBtu)',
                                                   'SiteEnergyUse(kBtu)',
                                                   'NaturalGas(kBtu)',
                                                   'NumberofBuildings'])])),
                ('regressor', Lasso())])

In [33]:
# fit model by using grid search
param_grid = {
    'regressor__alpha'          : [0.0001, 0.001, 0.01, 0.10, 1]
}
grid_search = GridSearchCV(pp, param_grid, cv=5, verbose=1, n_jobs=-1, refit=True, scoring='r2')
grid_search.fit(X, y)
print(grid_search.best_params_)
print('----------------')

# model scoring
best_model = grid_search.best_estimator_
print(f'Model\'s score : {best_model.score(X, y).round(3)}') #est R2 ? Bon va falloir revoir pipeline etc.

Fitting 5 folds for each of 5 candidates, totalling 25 fits
{'regressor__alpha': 1}
----------------
Model's score : 0.689


In [34]:
lasso_regressor = best_model.named_steps['regressor']

# Access the coefficients of the Lasso
lasso_coefficients = lasso_regressor.coef_

print("Lasso Coefficients:")
print(lasso_coefficients)

# Print number of total variables and total variables with non null parameter
(
    lasso_coefficients.shape,
    lasso_coefficients[lasso_coefficients != 0.].shape
)
# Le modele a simplifié notre equation: il a choisi seulement 14 variables parmis les 17 !

Lasso Coefficients:
[-0.00000000e+00  3.30171588e+03  1.12407265e+02 -0.00000000e+00
  3.02766217e+02 -1.75753305e+02 -0.00000000e+00 -8.10549978e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -1.58726204e+01
  0.00000000e+00  4.54366431e+01 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  8.83098501e+00  0.00000000e+00 -1.56855911e+01
  0.00000000e+00  0.00000000e+00  2.31865198e+01  1.91846001e+02
 -0.00000000e+00 -9.92940285e+00  1.13859383e+01  0.00000000e+00
 -1.05050342e-01 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  2.61266021e+02  3.00461929e+00  6.11935096e+01  5.73262635e+01]


((36,), (18,))